<center><b>Chatura Samarasinghe<br>Batch Gradient Descent vs. Autodiff Tensorflow and Tensorboard</b></center>

In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

In [2]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S") 
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [3]:
housing = fetch_california_housing()
m,n = housing.data.shape

adding a bias

In [4]:
import numpy as np
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

setup the variables

In [5]:
from sklearn.preprocessing import StandardScaler

def setup_variables():
    X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
    y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
    
    #normalizing data
    scaler = StandardScaler()
    scaled_housing_data = scaler.fit_transform(housing.data)
    scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]
    X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
    
    return X,y,scaled_housing_data_plus_bias

In [6]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

In [7]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

<b>Gradient Descent</b>

<b>Batch Gradient Descent</b>

In [8]:
tf.reset_default_graph()

In [9]:
X,y,scaled_housing_data_plus_bias = setup_variables()

In [10]:
n_epochs = 1000 
learning_rate = 0.01

# assigning random values to theta intially
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), dtype=tf.float32, name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [11]:
# Graph

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [12]:
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign (theta, theta - learning_rate * gradients)

In [13]:
# for displaying the graph
mse_summary = tf.summary.scalar('MSE_-_Batch_Gradient_Descent', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [14]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
    max_theta = theta.eval()

In [15]:
file_writer.flush()
file_writer.close()

Operations list

In [16]:
tf.get_default_graph().get_operations()

[<tf.Operation 'X' type=Const>,
 <tf.Operation 'y' type=Const>,
 <tf.Operation 'X_1' type=Const>,
 <tf.Operation 'random_uniform/shape' type=Const>,
 <tf.Operation 'random_uniform/min' type=Const>,
 <tf.Operation 'random_uniform/max' type=Const>,
 <tf.Operation 'random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'random_uniform/sub' type=Sub>,
 <tf.Operation 'random_uniform/mul' type=Mul>,
 <tf.Operation 'random_uniform' type=Add>,
 <tf.Operation 'theta' type=VariableV2>,
 <tf.Operation 'theta/Assign' type=Assign>,
 <tf.Operation 'theta/read' type=Identity>,
 <tf.Operation 'predictions' type=MatMul>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'Square' type=Square>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'mse' type=Mean>,
 <tf.Operation 'X_2' type=Placeholder>,
 <tf.Operation 'y_1' type=Placeholder>,
 <tf.Operation 'random_uniform_1/shape' type=Const>,
 <tf.Operation 'random_uniform_1/min' type=Const>,
 <tf.Operation 'random_uniform_1/max' type=Const>,
 <t

<b>Autodiff</b>

In [17]:
tf.reset_default_graph()

In [18]:
X,y, scaled_housing_data_plus_bias = setup_variables()

In [19]:
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

# assigning random values to theta intially
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), dtype=tf.float32, name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [20]:
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign (theta, theta - learning_rate * gradients)

In [21]:
mse_summary = tf.summary.scalar("MSE_-_Autodiff",mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [22]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
    max_theta = theta.eval()

In [23]:
file_writer.flush()
file_writer.close()

Operations List

In [24]:
tf.get_default_graph().get_operations()

[<tf.Operation 'X' type=Const>,
 <tf.Operation 'y' type=Const>,
 <tf.Operation 'X_1' type=Const>,
 <tf.Operation 'X_2' type=Placeholder>,
 <tf.Operation 'y_1' type=Placeholder>,
 <tf.Operation 'random_uniform/shape' type=Const>,
 <tf.Operation 'random_uniform/min' type=Const>,
 <tf.Operation 'random_uniform/max' type=Const>,
 <tf.Operation 'random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'random_uniform/sub' type=Sub>,
 <tf.Operation 'random_uniform/mul' type=Mul>,
 <tf.Operation 'random_uniform' type=Add>,
 <tf.Operation 'theta' type=VariableV2>,
 <tf.Operation 'theta/Assign' type=Assign>,
 <tf.Operation 'theta/read' type=Identity>,
 <tf.Operation 'predictions' type=MatMul>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'Square' type=Square>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'mse' type=Mean>,
 <tf.Operation 'gradients/Shape' type=Const>,
 <tf.Operation 'gradients/grad_ys_0' type=Const>,
 <tf.Operation 'gradients/Fill' type=Fill>,
 <tf.Operation 'gr